In [11]:
from typing import List, Optional
from pydantic import BaseModel, Field
from pydantic_ai import Agent, RunContext
from langchain_community.retrievers import WikipediaRetriever
from dotenv import load_dotenv
import os

load_dotenv()

True

Flow

Input topic -> Topic Expansion (list) \
Topic Expansion -> Research on relevant ropics (dict) \
Research on relevant topics -> Generate Outline (str) \
Generate Outline -> Generate Article (str) \


## Topics Expansion


In [39]:
class RelatedTopics(BaseModel):
    topics: List[str] = Field(description="A list of related topics with maximum of 3.")


# Create agent
topic_explorer_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=RelatedTopics,
    system_prompt=(
        "You are an experienced researcher. Given a research topic, return a list of relevant topics to research on."
        "Avoid returning topics that are too broad or too specific. "
        "The list of topics will then use to research on wikipedia to find relevant information. "
    ),
)


async def gen_related_topics(topic: str):
    related_topics = await topic_explorer_agent.run(topic)
    # related_topics_str = ", ".join(related_topics.data.topics)
    return related_topics.data.topics


topic = "AI Agents in Workforce."

related_topics = await gen_related_topics(topic)
print(related_topics)

['Impact of AI on Job Automation', 'Roles of AI in Human Resource Management', 'AI in Employee Training and Development']


## Research on the topics


In [40]:
from tavily import AsyncTavilyClient


class TavilySearch:
    def __init__(self, max_results: int = 3):
        TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
        self.tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
        self.max_results = max_results

    async def asearch(self, query: str) -> dict:
        results = {}
        response = await self.tavily_client.search(query, max_results=self.max_results)
        search_results = response["results"]
        for r in search_results:
            source = {r["url"]: r["content"]}
            results.update(source)
        return results

    async def abatch_search(self, queries: list[str]) -> dict:
        search_results = {}
        for query in queries:
            results = await self.asearch(query)
            search_results.update(results)
        return search_results

    @staticmethod
    def results_to_str(search_results: dict, max_len: int = 50_000) -> str:
        return "\n\n##########\n".join(
            f"URL:\n {url}\n\nContent:\n{content}"
            for url, content in search_results.items()
        )[:max_len]


# Usage
tavily_search = TavilySearch()
search_results = await tavily_search.abatch_search(related_topics)
search_results_str = TavilySearch.results_to_str(search_results)

print(search_results_str)

{'https://www.researchgate.net/publication/376522892_The_Impact_of_Artificial_Intelligence_on_Job_Automation': 'This journal focuses on the relationship between artificial intelligence(AI) and job automation. The study analyses the technological landscape.', 'https://www.forbes.com/sites/bernardmarr/2024/05/15/hype-or-reality-will-ai-really-take-over-your-job/': 'Experts predict that the impact of AI and automation on the job market will be significant – according to the WEF, 85 million jobs will be displaced.', 'https://ssir.org/articles/entry/ai-impact-on-jobs-and-work': 'The government should adopt a set of complementary policies encouraging a better path for technology use and development, as we detail in a recent policy memo (jointly written with our MIT colleague David Autor, through the new MIT Shaping the Future of Work Initiative that we collectively co-direct) entitled “Can We Have Pro-Worker AI?”\nLawmakers should revise the federal tax code to equalize the tax burden across

## Write Article


In [41]:
## Generate Outline
class Subsection(BaseModel):
    title: str = Field(description="The title of the subsection.")
    content: str = Field(description="The content of the subsection.")


class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    subtitles: List[Subsection] = Field(description="A list of subsections.")

    @property
    def as_str(self):
        subsections = "\n\n".join(
            f"### {s.title}\n\n {s.content}" for s in self.subtitles
        )
        return f"## {self.title}\n\n{self.content}\n\n{subsections}"


class Outline(BaseModel):
    title: str = Field(description="The title of the article.")
    sections: List[Section]

    @property
    def as_str(self):
        sections = "\n\n".join(f"{s.as_str}" for s in self.sections)
        return f"# {self.title}\n\n{sections}"


In [52]:
# Generate Article
gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher."
        "Based on the resouces gathered so far, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
)


example_article = await gen_article_agent.run(
    search_results_str,
    result_type=Outline,
)


## Chaining everything together


In [58]:
from pydantic_ai import Tool

gen_article_agent = Agent(
    "openai:gpt-4o-mini",
    result_type=Outline,
    deps_type=Optional[Outline],
    system_prompt=(
        "You are an experienced researcher, given a research topic, use gen_related_topics to find more relevant topics."
        "Use tavily_search.abatch_search tool to gather relevant information with the relevant topics."
        "Based on the resouces gathered, generate an article based on the outline."
        "Use only the information provided gathered anf cite the url of the source for each section."
        "Make sure the article is informative and cohesive."
    ),
    tools=[Tool(tavily_search.abatch_search), Tool(gen_related_topics)],
)

article = await gen_article_agent.run("local AI adoption")

In [59]:
from IPython.display import display, Markdown

display(Markdown(article.data.as_str))

#Local AI Adoption: Driving Innovation and Overcoming Challenges

## Factors Influencing Local AI Adoption

AI adoption at the local level is affected by a blend of factors categorized into individual, social, organizational, environmental, and technological elements. Studies indicate that the presence of transformational leadership can significantly facilitate the successful implementation of AI technologies in organizations. Transformational leaders foster a supportive environment that encourages innovation, making the adoption of AI more likely as they guide organizations through strategies that leverage their technical capabilities and digital maturity.

### Transformational Leadership and AI Adoption

 Transformational leadership plays a critical role in technology adoption, with leaders influencing intention and capability within organizations. Their competencies, such as vision and intellectual stimulation, are pivotal in fostering an atmosphere conducive to AI adoption, effectively improving an organization's readiness to engage with AI technologies, as highlighted in various studies. 

(https://www.sciencedirect.com/science/article/pii/S0160791X24002987)

## Impacts of AI on Local Economies

The advent of AI technologies is reshaping local economies by enhancing productivity, driving innovation, and generating new jobs. AI tools, such as machine learning and automation, are instrumental in revitalizing local businesses and spurring economic growth. The ability of AI to foster creativity and unlock untapped potential within communities has become increasingly evident, as cities that adopt AI experience profound transformations in their economic landscapes.

### AI as a Catalyst for Economic Growth

 AI is emerging as a key disruptor across various sectors, especially in local markets. It enables businesses to streamline operations and improve customer engagement, significantly enhancing overall efficiency. Additionally, AI's role in precision farming and automation augments productivity, which ultimately contributes to community wealth and job creation, showcasing how local economies can benefit from embracing AI technologies.

(https://www.ewadirect.com/proceedings/ace/article/view/16842)

## Challenges in Local AI Implementation

While the benefits of AI are clear, local governments and organizations often face substantial challenges in its implementation. Obstacles include a lack of knowledge and technical competence, concerns about data privacy, and the need for integration with existing systems. Furthermore, only about 46% of local government leaders feel prepared to effectively utilize AI tools, highlighting the need for targeted training and strategic planning in AI adoption initiatives.

### Barriers to Successful AI Implementation

 Key hurdles in AI implementation include challenges related to data collection, quality, and integration. Local governments must establish robust systems to manage data effectively while ensuring ethical considerations are met. Continuous employee training and collaboration with AI experts are crucial strategies for overcoming these challenges, as is a commitment to ethical transparency in AI use.

(https://elearningindustry.com/ai-implementation-challenges-and-how-to-overcome-them)

## Result

Based on the research results, update the initial outline.


In [54]:
from IPython.display import display, Markdown

display(Markdown(example_article.data.as_str))

#The Impact of Artificial Intelligence on Job Automation and Employee Development

## Introduction

Artificial Intelligence (AI) is rapidly reshaping the technological landscape, particularly in relation to job automation and employee development. Experts predict profound effects of AI on the job market, with significant job displacement anticipated. This article aims to explore these themes, focusing on both the challenges and opportunities presented by AI in the workplace.



## The Relationship Between AI and Job Automation

The integration of AI into various industries has initiated a widespread discussion about job automation. A recent study highlights that technological advancements are not just altering job roles but are also threatening the existence of many. According to the World Economic Forum (WEF), an estimated 85 million jobs may be displaced due to automation driven by AI technologies. This statistic underscores the urgency for governments, organizations, and workers to strategize for an uncertain future. Furthermore, experts warn that leaving these decisions in the hands of a few could exacerbate existing inequalities in the workforce, leading to a widening gap between capital and labor.



## Policy Recommendations for Embracing AI

To navigate the complexities of AI and job automation, it is suggested that governments implement policies that promote equitable technology use. For instance, revising the federal tax code to equalize tax burdens on labor and machines could incentivize companies to prioritize human workers over automated processes. There is an increasing need to bridge the gap created by automation, as evidenced by higher pro-union sentiments and collective actions demanding fair practices in utilizing AI technologies.



## AI in Human Resource Management

The role of AI extends beyond automation; it is transforming human resource management practices. HR professionals are increasingly utilizing AI to enhance various functions such as employee performance tracking, setting goals, and facilitating training. AI enables the creation of customized learning programs tailored to individual employee needs, leading to improved job performance and retention of skills. This shift not only benefits employees by supporting their professional growth but also optimizes organizational operations by maximizing human resource potential.

### Personalized Training and Development

 AI technologies are being integrated into employee training and development frameworks, allowing for personalized feedback and assessment of individual learning styles. The ability to adapt training programs based on specific needs can lead to more effective skill acquisition and application in real-world scenarios. Organizations that leverage AI for training are likely to see a significant return on investment through improved employee performance and engagement.

## Conclusion

In conclusion, the impact of AI on job automation and employee development is profound and multifaceted. While concerns regarding job displacement are valid, it is essential to recognize the potential of AI to enhance human capabilities and promote equitable workplace practices. As organizations navigate this evolving landscape, implementing thoughtful policies and embracing technological advancements will be crucial in fostering a productive and inclusive work environment.



## References

1. The Impact of Artificial Intelligence on Job Automation. Retrieved from [ResearchGate](https://www.researchgate.net/publication/376522892_The_Impact_of_Artificial_Intelligence_on_Job_Automation)  
2. Hype or Reality: Will AI Really Take Over Your Job? Retrieved from [Forbes](https://www.forbes.com/sites/bernardmarr/2024/05/15/hype-or-reality-will-ai-really-take-over-your-job/)  
3. AI Impact on Jobs and Work. Retrieved from [SSIR](https://ssir.org/articles/entry/ai-impact-on-jobs-and-work)  
4. What AI in Human Resource Management. Retrieved from [Retorio](https://www.retorio.com/blog/what-ai-in-human-resource-management)  
5. AI in HR. Retrieved from [Jobylon](https://www.jobylon.com/blog/ai-in-hr/) 
6. The Role of AI in Employee Training and Development. Retrieved from [The Data Scientist](https://thedatascientist.com/the-role-of-ai-in-employee-training-and-development/)  
7. The Benefits of Artificial Intelligence in Employee Training and Development. Retrieved from [Business Tech Weekly](https://www.businesstechweekly.com/hr-and-recruitment/learning-development/artificial-intelligence-in-training-and-development/)  
8. AI in Employee Training and Development. Retrieved from [Springworks](https://www.springworks.in/blog/ai-in-employee-training-and-development/)



In [55]:
article.all_messages()

[ModelRequest(parts=[SystemPromptPart(content='You are an experienced researcher.Based on the resouces gathered so far, generate an article based on the outline.Make sure the article is informative and cohesive.', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='local AI adoption', timestamp=datetime.datetime(2025, 1, 15, 8, 29, 21, 542653, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='gen_related_topics', args=ArgsJson(args_json='{"topic":"local AI adoption"}'), tool_call_id='call_L8XTTk5NQZy6TS274ViGHb7J', part_kind='tool-call')], timestamp=datetime.datetime(2025, 1, 15, 8, 29, 22, tzinfo=datetime.timezone.utc), kind='response'),
 ModelRequest(parts=[ToolReturnPart(tool_name='gen_related_topics', content=['Challenges of local AI adoption', 'Impact of local regulations on AI implementation', 'Community-driven AI initiatives'], tool_call_id='call_L8XTTk5NQZy6TS274ViGHb7J', timestamp=datetime